## Transferencia de aprendizaje 
La idea del concepto de transferencia de aprendizaje o [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning) es utilizar conceptos (o los parámetros de la red) aprendidos en un problema en otro problema.

Particularmente en redes convolucionales, se reutilizan los pesos de la parte de extracción características o feature extraction, correspondientes con las capas convolucionales y de pooling, reentrenando normalmente los pesos de la parte de clasificación, es decir, las últimas capas densas.

![](img/cnn.jpeg)

### Problema a resolver: Clasificación binaria de imágenes de fondo de ojo para detectar retinopatía diabética

Se pretende resolver este problema utilizando la técnica de transferencia de aprendizaje, con el fin de realizar una clasificación binaria de las imágenes de fondo de ojo para detectar [retinopatía diabética](https://es.wikipedia.org/wiki/Retinopat%C3%ADa_diab%C3%A9tica).
Por lo que tenemos 5 niveles de severidad repartidos en 2 categorías para realizar una clasificación binaria
![](img/image1.png)

Queremos utilizar redes con parámetros entrenados previamente en otro problema, para esto existen redes preentrenadas en la base de datos [ImageNet](http://www.image-net.org/), las cuales pueden ser cargadas desde [Keras](https://keras.io/api/applications/). Las redes estan preentrenadas para realizar la clasificación de las imágenes de la base de datos ImageNet, las cuales se encuentran clasificadas en 1000 categorías.


  
![](img/imagenet.jpeg)

Utilizaremos 3 de estas redes preentrenadas: Xception, Inception V3 y DenseNet 169, cuyas principales características se muestran a continuación
<img src="img/networks.png" width="700">

Las imágenes de fondo de ojo fueron obtenidas de la base de datos [Kaggle: Diabetic retinopathy detection](https://www.kaggle.com/c/diabetic-retinopathy-detection)
El histograma de las clases se muestra a continuación que la cantidad de imágenes para cada nivel de severidad de la enfermedad se encuentra muy desbalanceada. Sin embargo, solo vamos a usar 5000 imágenes, con categorías balanceadas.
<img src="img/histogram.png" width="500">


Como se puede ver, el problema de detección de retinopatía diabética no parece un problema sencillo, por lo que 
antes de resolver el problema con todos los niveles de severidad de la enfermedad, resolvemos un problema **más simple**, clasificando solo imágenes de personas sanas y del mayor nivel de severidad de la enfermedad.
![](img/image6.png)

Antes de usar redes convolucionales, intentamos resolver el problema con métodos más simples, como **Random Forest, SVM, MLP y KNN**. Los resultados de precisión obtenida son:
<img src="img/metodos.png" width="300">
Se observa que la precisión obtenida es prácticamente la aleatoria, por lo que procedemos a usar redes convolucionales

Usamos por lo tanto CNN, todavía intentando resolver el problema simple de **solo 2 niveles**, intentando ver si funcionan las redes convolucionales para este problema.
Vemos primero las tasas de aprendizajes y la constante de regularización apropiadas
<img src="img/lr.png" width="700">


Luego vemos si los métodos de regularización son necesarios, usando dropout y regularización L2:
<img src="img/regularizacion.png" width="700">
Se observa que los métodos de regularización hacen que sea menos errático el aprendizaje y permiten obtener en menor cantidad de épocas mejores valores de precisión en el set de validación

Vimos que para aplicar transferencia de aprendizaje hay que reentrenar los pesos de la parte de clasificación de la red (las últimas capas densas), pero además es conveniente entrenar parte de los pesos de las capas convolucionales. Hasta que profundidad se entrena o cuales de estas capas es conveniente entrenar?
## Red Inception V3
<img src="img/arquitectura.png" width="700">
Observamos que las capas convolucionales tienen una estructura en bloques

Se hace por lo tanto Fine tuning (o reentrenamiento de los pesos) hasta varias profundidades, donde los puntos rojos y negros corresponden a entrenar hasta bloque enteros de la parte de capas convolucionales
<img src="img/profundidad.png" width="700">
Por lo que se observa que la mayor precisión se obtiene entrenando hasta bloques enteros, en lugar de entrenar hasta capas intermedias.

Ahora que tenemos idea del valor de todos los hiperparámetros, podemos intentar resolver el problema completo con todos los niveles de severidad

<img src="img/image1.png" width="700">

Se obtuvieron las siguiente precisiones con imágenes de tamaño 224x224 para las 3 redes utilizadas
<img src="img/resultados1.png" width="400">


Qué comclusiones podemos sacar de estas soluciones obtenidas?
Se puede visualizar las regiones de importancia a la hora de clasificar? 

El método de [mapas de activación de clases](http://cnnlocalization.csail.mit.edu/) permite obtener las regiones de importancia a la hora de clasificar una imágen, sin embargo, solo funciona para redes de arquitectura particular
<img src="img/CAM.png" width="700">

El método [Guided Grad CAM](http://gradcam.cloudcv.org/) por otro lada funcion para arquitecturas más custom y además da los pixeles importantes, por lo que se tiene mayor definición
<img src="img/Guided_cam.png" width="700">

Se aplicaron los métodos **CAM** y **Guided Grad CAM** a las soluciones obtenidas, observando las regiones de importancia en la clasificación
<img src="img/Xception_224.png" width="1100">


Se aplicaron los métodos **CAM** y **Guided Grad CAM** a las soluciones obtenidas, observando las regiones de importancia en la clasificación

<img src="img/Densenet_224.png" width="1100">

Qué pasa si entrenamos redes con tamaños de imagen más grande?
<img src="img/solucion2.png" width="600">
La precisión mejora en todos los casos, por qué?

Aplicamos los métodos **CAM** y **Guided grad CAM** a las soluciones con imágenes más grandes
<img src="img/Xception_448.png" width="1100">

Aplicamos los métodos **CAM** y **Guided grad CAM** a las soluciones con imágenes más grandes
<img src="img/DenseNet_448.png" width="1100">

Tomamos una imágen que antes era clasificada mal y ahora bien, y le aplicamos los métodos de visualización

### DenseNet 224x224
<img src="img/not_ok.png" width="1000">

### DenseNet 448x448
<img src="img/red_ok.png" width="1000">

se observa que las lesiones no son detectadas por la red entrenadas con imágenes pequeñas, mientras que si lo son por la red con imágenes de mayor tamaño.